In [21]:
import pandas as pd
import numpy as np

In [22]:
# количество итераций обучения
NUM_OF_ITERATIONS = 400

# читаем размеченные данные из файла
beer_dataset = pd.read_excel('beer_data_set.xlsx')

# печатаем список колонок таблички чтобы удобнее было копировать
print('\n'.join(beer_dataset.columns))

# выводим несколько первых строк таблицы
beer_dataset.head()

barcode
SKU_NAME
Наименование
Алкоголь
Объем
Производитель
Бренд
Саб-бренд
Тип упаковки
Мультипак
тип


,barcode,SKU_NAME,Наименование,Алкоголь,Объем,Производитель,Бренд,Саб-бренд,Тип упаковки,Мультипак,тип
0,4600721002206,Пиво БагБир 0.5л ст/бут,BAGBIER - светлое - 4.2% - 0.5л стекло,0.042,0.5,AB InBev,BAGBIER,NaN,Стекло,NaN,Светлое
1,4600721003197,Пиво БАГ-БИР св.ст/б 0.5л,BAGBIER - светлое - 4.2% - 0.5л стекло,0.042,0.5,AB InBev,BAGBIER,NaN,Стекло,NaN,Светлое
2,4600721003203,"Пиво BAGBIER светлое 4,9% 1.5л",BAGBIER - светлое - 4.9% - 1.5л пэт,0.049,1.5,AB InBev,BAGBIER,NaN,ПЭТ,NaN,Светлое
3,4600721005191,Пиво БАГ-БИР св.ПЭТ 2.5л,BAGBIER - светлое - 4.2% - 2.5л пэт,0.042,2.5,AB InBev,BAGBIER,NaN,ПЭТ,NaN,Светлое
4,4600721009366,"Пиво БАГ БИР ГОЛЬДЕН светлое ПЭТ 4% 1,5л",BAGBIER GOLDEN - светлое - 4.7% - 1.5л пэт,0.047,1.5,AB InBev,BAGBIER,Golden,ПЭТ,NaN,Светлое


Импортируем преобразователь текста в вектор.

[Примерное объяснение как работает](http://zabaykin.ru/?p=463)

[Документация](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

Импортируем преобразователь меток в числа. 
Он присваивает каждому новой встреченной метке уникальный номер.

Допустим у нас есть такой набор меток:
```python
labels = ['заяц', 'волк', 'утка', 'белка', 'заяц', 'утка']
```

LabelEncoder строит примерно такое соответсвие:

| Метка | Номер |
|:-----:|:-----:|
|  волк |   0   |
| белка |   1   |
| заяц  |   2   |
| утка  |   3   |

Так что если к нам поступает такой набор меток,
```python
['утка', 'утка', 'заяц', 'утка', 'белка', 'волк', 'заяц', 'волк', 'волк', 'белка']
```
то мы можем преобразовать их в список чисел
```python
[3, 3, 2, 3, 1, 0, 2, 0, 0, 1]
```

Это нужно потому что многие алгоритмы не умеют работать со строками, к тому же числа занимаю меньше места

[Документация](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

In [24]:
from sklearn.preprocessing import LabelEncoder

Импортируем функцию для разбиения датасета на обучающий и тестирующий набор данных случайным образом. 

Зачем это нужно: когда модель обучается, то она может начать запоминать сочетания "вопрос"-"ответ". Вместо того, чтобы пытаться разобраться в во входных данных. Поэтому модель нужно проверять на данных, которых она до этого не видела. 

[Немного подробнее](http://robotosha.ru/algorithm/training-set-and-test-data.html)

[Документация](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [25]:
from sklearn.model_selection import train_test_split

Импортируем классификатор данных из библиотеки CatBoost от Яндекса

In [26]:
from catboost import CatBoostClassifier

Выдираем сырые строки из колонки **SKU_NAME**. В таблице строки хранятся в виде списков байт, метод `.flatten()` преобразует их в простые строки

In [27]:
data = [item for item in beer_dataset[['SKU_NAME']].values.flatten()]

Выдираем метки из колонки **Тип упаковки**. В таблице строки хранятся в виде списков байт, метод `.flatten()` преобразует их в простые строки. Так же на всякий случай переводим метки в нижний регистр с помощью метода `.lower()`

In [28]:
labels = [item.lower() for item in beer_dataset['Тип упаковки'].values.flatten()]

Разбиваем датасет случайным образом на обучающие и тестовые данные с соотношением 2 к 1. `random_state` – это начальное состояние генератора случайных чисел для разбиения, ставим число 42 чтобы каждый раз разбиение было одним и тем же.

In [29]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.33, random_state=42)

Печатаем статистику полученных датасетов: суммарный размер (Total), размер обучающей выборки (train) и тестовой (test)

In [30]:
print(f"Total: {len(data)} samples\n"
     f"\ttrain: {len(train_data)} data, {len(train_labels)} labels\n"
     f"\ttest: {len(test_data)} data, {len(test_labels)} labels")

Total: 5824 samples
	train: 3902 data, 3902 labels
	test: 1922 data, 1922 labels


Воспомогательная функция. CountVectorizer возвращает сжатые вектора, а нам нужны обычные. Эта функция берёт список сжатых веткоров и преобразует их в массивы чисел.

In [31]:
def dense_vectors(vectors):
    return [np.asarray(item.todense())[0] for item in vectors]

Функция обучения модели. Наша модель состоит из трёх частей:
    - `CountVectorizer` для преобразования входных данных в векторное представление
    - `LabelEncoder` для преобразования меток в числа
    - `CatBoostClassifier` – собственно классификатор данных

По-хорошему надо сохранять все три части в один или несколько файлов, пока для тестирования сохраняем только последнее. 

Входные данные в виде списка строк переводятся в нижний регистр, разбиваются на токены с помощью регулярного выражения `(?u)\b\w\w+\b|[0-9\.,]+[\%\w]|\w+`. Посмотреть как работает это выражение можно посмотреть [здесь](https://regex101.com/r/Puyk9J/1). Оно разбивает строки на список подстрок примерно так


|Строка| Токены |
|:------------------------------|:-----------------------------------:|
| Пиво БагБир 0.5л  ст/бут | ['Пиво', 'БагБир', '0.5л', 'ст', 'бут'] |
| Пиво БАГ-БИР св.ст/б 0.5л | ['Пиво', 'БАГ', 'БИР', 'св', '.с', 'т', 'б', '0.5л'] |
| Пиво BAGBIER светлое 4,9%   1.5л | ['Пиво', 'BAGBIER', 'светлое', '4,9%', '1.5л'] |
| Пиво БАГ-БИР св.ПЭТ 2.5л | ['Пиво', 'БАГ', 'БИР', 'св', '.П', 'ЭТ', '2.5л'] |
| Пиво БАГ БИР ГОЛЬДЕН светлое ПЭТ 4% 1,5л | ['Пиво', 'БАГ', 'БИР', 'ГОЛЬДЕН', 'светлое', 'ПЭТ', '4%', '1,5л'] |
| Пиво БАГ-БИР ГОЛЬДЕН св.4% ПЭТ 2.5л | ['Пиво', 'БАГ', 'БИР', 'ГОЛЬДЕН', 'св', '.4%', 'ПЭТ', '2.5л'] |
| Пиво БАГ БИР БОК темное 4% 1.5л | ['Пиво', 'БАГ', 'БИР', 'БОК', 'темное', '4%', '1.5л'] |
| Нап.пивн.BAGBIER 4,6% ст/б 0.5л | ['Нап', '.п', 'ивн', '.B', 'AGBIER', '4,6%', 'ст', 'б', '0.5л'] |
| Нап.пивн.БАГ БИР 4,2% ст/б 0.5л | ['Нап', '.п', 'ивн', '.Б', 'АГ', 'БИР', '4,2%', 'ст', 'б', '0.5л'] |
| Пиво BRAHMA 4.6% св.ст/б  0.33л | ['Пиво', 'BRAHMA', '4.6%', 'св', '.с', 'т', 'б', '0.33л'] |

Потом мы переводим данные в сжатые вектора, а из них получаем простые вектора с помощью фунции `dense_vectors`. Затем создаём LabelEncoder() и заставляем его запомнить наши метки.

Теперь можно заняться самым главным: обучить классификатор. 

Мы создаём CatBoostClassifier с настраиваемым параметром iterations (число итераций) и обучаем его через вызов метода `fit` ([документация](https://catboost.ai/docs/concepts/python-reference_catboostclassifier_fit.html)). Этот метод принимает обучающие данные в виде списка векторов (`vectorized_data`) и список закодированных меток.

После того, как обучились, возвращаем кортеж вида `(CountVectorizer, LabelEncoder, CatBoostClassifier)`

In [32]:
def build_model(data, labels, iterations=200):
        vectorizer = CountVectorizer(lowercase=True, 
                                     analyzer="word",
                                    token_pattern=r"(?u)\b\w\w+\b|[0-9\.,]+[\%\w]|\w+")
        
        compressed_data = vectorizer.fit_transform(data)
        vectorized_data = dense_vectors(compressed_data)
        
        le = LabelEncoder()
        encoded_labels = le.fit_transform(labels)
            
        classifier = CatBoostClassifier(iterations=iterations, task_type = "GPU")
        classifier.fit(vectorized_data, encoded_labels, silent=False)
        
        return (vectorizer, le, classifier)


Обучаем нашу модель

In [33]:
model = build_model(train_data, train_labels, iterations=NUM_OF_ITERATIONS)

0:	learn: -1.5139364	total: 26.2ms	remaining: 10.4s
1:	learn: -1.4297613	total: 44.8ms	remaining: 8.91s
2:	learn: -1.3553701	total: 68ms	remaining: 9s
3:	learn: -1.2889445	total: 90.1ms	remaining: 8.92s
4:	learn: -1.2291322	total: 108ms	remaining: 8.54s
5:	learn: -1.1753841	total: 126ms	remaining: 8.27s
6:	learn: -1.1257986	total: 155ms	remaining: 8.69s
7:	learn: -1.0803535	total: 174ms	remaining: 8.55s
8:	learn: -1.0385215	total: 192ms	remaining: 8.32s
9:	learn: -0.9998674	total: 211ms	remaining: 8.23s
10:	learn: -0.9640307	total: 230ms	remaining: 8.12s
11:	learn: -0.9321800	total: 249ms	remaining: 8.05s
12:	learn: -0.9009999	total: 266ms	remaining: 7.93s
13:	learn: -0.8728040	total: 282ms	remaining: 7.78s
14:	learn: -0.8471109	total: 304ms	remaining: 7.79s
15:	learn: -0.8211580	total: 321ms	remaining: 7.7s
16:	learn: -0.7976255	total: 337ms	remaining: 7.59s
17:	learn: -0.7745975	total: 353ms	remaining: 7.49s
18:	learn: -0.7519505	total: 370ms	remaining: 7.42s
19:	learn: -0.7312746	to

168:	learn: -0.1845323	total: 3.43s	remaining: 4.68s
169:	learn: -0.1842440	total: 3.44s	remaining: 4.66s
170:	learn: -0.1836624	total: 3.46s	remaining: 4.63s
171:	learn: -0.1832332	total: 3.47s	remaining: 4.61s
172:	learn: -0.1828042	total: 3.49s	remaining: 4.58s
173:	learn: -0.1825653	total: 3.5s	remaining: 4.55s
174:	learn: -0.1824170	total: 3.52s	remaining: 4.52s
175:	learn: -0.1822706	total: 3.53s	remaining: 4.5s
176:	learn: -0.1821394	total: 3.55s	remaining: 4.47s
177:	learn: -0.1816871	total: 3.56s	remaining: 4.45s
178:	learn: -0.1812972	total: 3.58s	remaining: 4.42s
179:	learn: -0.1811975	total: 3.59s	remaining: 4.39s
180:	learn: -0.1810920	total: 3.61s	remaining: 4.37s
181:	learn: -0.1809577	total: 3.63s	remaining: 4.34s
182:	learn: -0.1808499	total: 3.64s	remaining: 4.32s
183:	learn: -0.1806332	total: 3.66s	remaining: 4.29s
184:	learn: -0.1801371	total: 3.67s	remaining: 4.27s
185:	learn: -0.1797452	total: 3.69s	remaining: 4.25s
186:	learn: -0.1796229	total: 3.7s	remaining: 4.

324:	learn: -0.1595587	total: 5.99s	remaining: 1.38s
325:	learn: -0.1595276	total: 6.02s	remaining: 1.37s
326:	learn: -0.1593526	total: 6.04s	remaining: 1.35s
327:	learn: -0.1592437	total: 6.05s	remaining: 1.33s
328:	learn: -0.1590725	total: 6.07s	remaining: 1.31s
329:	learn: -0.1590341	total: 6.08s	remaining: 1.29s
330:	learn: -0.1590013	total: 6.1s	remaining: 1.27s
331:	learn: -0.1588138	total: 6.11s	remaining: 1.25s
332:	learn: -0.1587659	total: 6.13s	remaining: 1.23s
333:	learn: -0.1587294	total: 6.14s	remaining: 1.21s
334:	learn: -0.1585648	total: 6.16s	remaining: 1.19s
335:	learn: -0.1585218	total: 6.17s	remaining: 1.18s
336:	learn: -0.1583910	total: 6.19s	remaining: 1.16s
337:	learn: -0.1583547	total: 6.21s	remaining: 1.14s
338:	learn: -0.1582157	total: 6.22s	remaining: 1.12s
339:	learn: -0.1581837	total: 6.24s	remaining: 1.1s
340:	learn: -0.1581427	total: 6.25s	remaining: 1.08s
341:	learn: -0.1580706	total: 6.28s	remaining: 1.06s
342:	learn: -0.1579142	total: 6.32s	remaining: 1

Генератор отчёта по нашей модели. Скармливаем ей модель, тестовые данные и метки.

Метод `.predict` ([документация](https://catboost.ai/docs/concepts/python-reference_catboostclassifier_predict.html)) классификатора `CountVectorizer` принимает список векторов (входные данные) и возвращает для них самые вероятные ответы в виде чисел float, которые нужно преобразовать к целым числам.

Метод `.predict_proba` ([документация](https://catboost.ai/docs/concepts/python-reference_catboostclassifier_predict_proba.html)) классификатора `CountVectorizer` принимает список векторов (входные данные) и возвращает для них вероятности полученных выше ответов. Полученные вероятности умножем на 100 и преобразумем в строки вида `95%, 80%, 91%`. 

После этого берём полученные из метода `.predict` ответы и раскодируем их с помощью метода `.inverse_transform` кодировщика `LabelEncoder`, который преобразует список чисел в список строк.

Из полученных выше ответов, входных данных и правильных ответов из тестового датасета делаем табличку (`table`) и преобразуем её в удобный для нас вид `pandas.DataFrame`.

In [34]:
def validate_model(model, valid_data, valid_labels):
    vectorizer, le, classifier = model
    columns = ["Запись", "Предсказание", "Уверенность" ,"Правильный результат"]

    compressed_data = vectorizer.transform(valid_data)
    vectorized_data = dense_vectors(compressed_data)
    prediction = classifier.predict(vectorized_data).flatten().astype('int64')
    
    proba = np.rint(100*classifier.predict_proba(vectorized_data).max(axis=1))
    proba_column = (f"{int(item)}%" for item in proba)
    
    results = le.inverse_transform(prediction).flatten()
    table = zip(valid_data, results, proba_column, valid_labels)
    return pd.DataFrame(table, columns=columns)

Запускаем валидацию нашей модели и выводим сравнительную табличку результатов

In [35]:
validation = validate_model(model, test_data, test_labels)
validation.head(20)

,Запись,Предсказание,Уверенность,Правильный результат
0,"Пиво СТЕПАН РАЗИН ПЕТР.св.4,7% ПЭТ 1.4л",пэт,97%,пэт
1,ZLATY BAZANT - светлое 4.1% - 0.5л ж.б.,ж.б.,98%,ж.б.
2,"Пиво MILLER MIDNIGHT тем.4,8% ст/б 0.5л",стекло,97%,стекло
3,КЛИНСКОЕ - светлое 4.5% - 0.5л стекло,стекло,97%,стекло
4,"ПивоANDECH.ВАЙСБ.ХЕФЕТР.св.5,5%ст/б 0.5л",стекло,97%,стекло
5,ЖИГУЛЕВСКОЕ - светлое 4.2% - 1.5л пэт,пэт,97%,пэт
6,"Пиво ЯРПИВО КРЕПК.св. 7,2% 0.5л",стекло,91%,ж.б.
7,БАЛТИКА - светлое 5.4% - 0.5л стекло,стекло,97%,стекло
8,SIBIRSKAYA KORONA(AB INBEV) KLASSICHESKOE 0.5L...,ж.б.,92%,ж.б.
9,OKHOTA(HEINEKEN BEL) KREPKOE EKSPORTNOE 1.4L P...,пэт,94%,пэт


Хорошо бы посчитать процент правильных ответов. Фильтруем все строки, в которых значение в столбце **Предсказание** совпадает со значением в столбце **Правильный результат** и считаем их количество.

Делим число правильных ответов на размер тестового набора данных и выводим.

In [36]:
valid = len(validation[validation['Предсказание'] == validation['Правильный результат']])
total = len(validation)
print(f"Valid: {valid} from {total} ({100*valid/total}%)")


Valid: 1829 from 1922 (95.16129032258064%)


Не забываем сохранить модель в файл!

In [37]:
vectorizer, le, classifier = model

model_name = f"beer_container_catboost_{NUM_OF_ITERATIONS}"

classifier.save_model(f"{model_name}.cbm")

In [46]:
from joblib import dump, load

dump(le, f"{model_name}_le.job")
dump(vectorizer, f"{model_name}_vect.job")

['beer_container_catboost_400_vect.job']